In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from boostedhh import utils, hh_vars

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
MAIN_DIR = Path("../../../")

plot_dir = MAIN_DIR / "plots/TriggerStudy/24Nov8"
plot_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
df = pd.read_parquet(
    "/ceph/cms/store/user/rkansal/bbtautau/skimmer/24Nov7SignalStudy_v12_private_signal/parquet/"
)

In [ ]:
df

In [ ]:
ptaus = df["ak8FatJetParTXtauhtauh"] + df["ak8FatJetParTXtauhtaue"] + df["ak8FatJetParTXtauhtaum"]
plt.hist(ptaus[1], np.linspace(0, 1, 100))
# df["ak8FatJetParTXtauhtauh"]

In [ ]:
df4b = pd.read_parquet(
    "/ceph/cms/store/user/rkansal/bbtautau/skimmer/24Nov7SignalStudy_v12_private_signal/2022/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_LHEweights_TuneCP5_13p6TeV_powheg-pythia8/parquet"
)

In [ ]:
cf = utils.get_cutflow(
    "/ceph/cms/store/user/rkansal/bbtautau/skimmer/24Nov7SignalStudy_v12_private_signal/pickles/",
    "2022",
    "GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_LHEweights_TuneCP5_13p6TeV_powheg-pythia8",
)

In [ ]:
cf4b = utils.get_cutflow(
    "/ceph/cms/store/user/rkansal/bbtautau/skimmer/24Nov7SignalStudy_v12_private_signal/2022/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_LHEweights_TuneCP5_13p6TeV_powheg-pythia8/pickles/",
    "2022",
    "GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_LHEweights_TuneCP5_13p6TeV_powheg-pythia8",
)

In [ ]:
cf["ak8_pt"] / cf["all"]

In [ ]:
cf4b["ak8_pt"] / cf4b["all"]

In [ ]:
np.mean((df["ak8FatJetPt"][1] > 250))

In [ ]:
np.mean((df4b["ak8FatJetPt"][1] > 250))

In [ ]:
np.sum((df["ak8FatJetPt"][1] > 250) * df["GenTauhe"][0]) / np.sum(df["GenTauhe"][0])

In [ ]:
# Tau triggers Run 3 https://twiki.cern.ch/twiki/bin/view/CMSPublic/Run3TauHLT

triggers_dict = {
    "tauhh": {
        "mask": df["GenTauhh"][0],
        "label": r"$\tau_h\tau_h$",
        "triggers": {
            "ParticleNetBB": df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0],
            "Di-tau": df["HLT_DoubleMediumDeepTauPFTauHPS35_L2NN_eta2p1"][0]
            | df["HLT_DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet60"][0]
            | df["HLT_DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet75"][0],
            "ParticleNetBB | PFJet425": (
                df["HLT_AK8PFJet425_SoftDropMass40"][0]
                | df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
            ),
            "ParticleNetBB | PFJet425 | Di-tau": (
                df["HLT_AK8PFJet425_SoftDropMass40"][0]
                | df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
                | df["HLT_DoubleMediumDeepTauPFTauHPS35_L2NN_eta2p1"][0]
                | df["HLT_DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet60"][0]
                | df["HLT_DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet75"][0]
            ),
        },
    },
    "tauhmu": {
        "mask": df["GenTauhmu"][0],
        "label": r"$\tau_h\mu$",
        "triggers": {
            "ParticleNetBB": df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0],
            "IsoMu24 | Mu50": (df["HLT_IsoMu24"][0] | df["HLT_Mu50"][0]),
            "ParticleNetBB | IsoMu24 | Mu50": (
                df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
                | df["HLT_IsoMu24"][0]
                | df["HLT_Mu50"][0]
            ),
            "ParticleNetBB | IsoMu24 | Mu50 | Mu-tau": (
                df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
                | df["HLT_IsoMu24"][0]
                | df["HLT_Mu50"][0]
                | df["HLT_IsoMu20_eta2p1_LooseDeepTauPFTauHPS27_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_LooseDeepTauPFTauHPS180_eta2p1"][0]
                | df["HLT_IsoMu24_eta2p1_LooseDeepTauPFTauHPS30_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS35_L2NN_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS45_L2NN_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_TightID_CrossL1"][0]
            ),
            "ParticleNetBB | IsoMu24 | Mu50 | Mu-tau+Jet": (
                df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
                | df["HLT_IsoMu24"][0]
                | df["HLT_Mu50"][0]
                | df["HLT_IsoMu20_eta2p1_LooseDeepTauPFTauHPS27_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_LooseDeepTauPFTauHPS180_eta2p1"][0]
                | df["HLT_IsoMu24_eta2p1_LooseDeepTauPFTauHPS30_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS35_L2NN_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS45_L2NN_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_CrossL1"][0]
                | df["HLT_IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_TightID_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet60_CrossL1"][0]
                | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet75_CrossL1"][0]
            ),
        },
    },
    "tauhe": {
        "mask": df["GenTauhe"][0],
        "label": r"$\tau_he$",
        "triggers": {
            "ParticleNetBB": df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0],
            "Electron": (
                df["HLT_Ele30_WPTight_Gsf"][0]
                | df["HLT_Ele115_CaloIdVT_GsfTrkIdT"][0]
                | df["HLT_Ele50_CaloIdVT_GsfTrkIdT_PFJet165"][0]
                # | df["HLT_Photon200"][0]
            ),
            "ParticleNetBB | Electron": (
                df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
                | df["HLT_Ele30_WPTight_Gsf"][0]
                | df["HLT_Ele115_CaloIdVT_GsfTrkIdT"][0]
                | df["HLT_Ele50_CaloIdVT_GsfTrkIdT_PFJet165"][0]
                # | df["HLT_Photon200"][0]
            ),
            "ParticleNetBB | Electron | E-tau": (
                df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
                | df["HLT_Ele30_WPTight_Gsf"][0]
                | df["HLT_Ele115_CaloIdVT_GsfTrkIdT"][0]
                | df["HLT_Ele50_CaloIdVT_GsfTrkIdT_PFJet165"][0]
                | df["HLT_Photon200"][0]
                | df["HLT_Ele24_eta2p1_WPTight_Gsf_LooseDeepTauPFTauHPS30_eta2p1_CrossL1"][0]
                | df["HLT_Ele24_eta2p1_WPTight_Gsf_TightChargedIsoPFTauHPS30_eta2p1_CrossL1"][0]
            ),
        },
    },
}

In [ ]:
twoboosted = df["ak8FatJetPt"][1] > 300

In [ ]:
mask = triggers_dict["tauhh"]["mask"]
print(
    np.sum(triggers_dict["tauhh"]["triggers"]["ParticleNetBB | PFJet425 | Di-tau"][mask])
    / np.sum(mask)
)
print(
    np.sum(
        ((triggers_dict["tauhh"]["triggers"]["ParticleNetBB | PFJet425 | Di-tau"]) * twoboosted)[
            mask
        ]
    )
    / np.sum((mask * twoboosted))
)

In [ ]:
mask = triggers_dict["tauhmu"]["mask"]
print(
    np.sum(triggers_dict["tauhmu"]["triggers"]["ParticleNetBB | IsoMu24 | Mu50 | Mu-tau+Jet"][mask])
    / np.sum(mask)
)
print(
    np.sum(
        (
            (triggers_dict["tauhmu"]["triggers"]["ParticleNetBB | IsoMu24 | Mu50 | Mu-tau+Jet"])
            * twoboosted
        )[mask]
    )
    / np.sum((mask * twoboosted))
)

In [ ]:
mask = triggers_dict["tauhe"]["mask"]
print(
    np.sum(triggers_dict["tauhe"]["triggers"]["ParticleNetBB | Electron | E-tau"][mask])
    / np.sum(mask)
)
print(
    np.sum(
        ((triggers_dict["tauhe"]["triggers"]["ParticleNetBB | Electron | E-tau"]) * twoboosted)[
            mask
        ]
    )
    / np.sum((mask * twoboosted))
)

In [ ]:
weights = df["weight"][0]

In [ ]:
higgs = utils.make_vector(df, name="GenHiggs")
mhh = (higgs[:, 0] + higgs[:, 1]).mass

In [ ]:
plt.rcParams.update({"font.size": 14})

for cat, vals in triggers_dict.items():
    (mask, label, triggers) = vals.values()

    fig, (ax, rax) = plt.subplots(
        2, 1, figsize=(12, 14), gridspec_kw=dict(height_ratios=[4, 1], hspace=0.07), sharex=True
    )

    bins = np.linspace(250, 1500, 30)
    hists = {"Preselection": np.histogram(mhh[mask], bins=bins, weights=weights[mask])}
    ratios = {}

    hep.histplot(
        hists["Preselection"],
        yerr=False,
        label="Preselection",
        ax=ax,
    )

    colours = plt.cm.tab10.colors[1:]

    for key, c in zip(triggers.keys(), colours):
        print(key)
        hists[key] = np.histogram(
            mhh[mask & triggers[key]], bins=bins, weights=weights[mask & triggers[key]]
        )
        ratios[key] = hists[key][0] / hists["Preselection"][0]

        hep.histplot(
            hists[key],
            yerr=False,
            label=key,
            ax=ax,
            color=c,
        )

        hep.histplot(
            (ratios[key], bins),
            yerr=False,
            label=key,
            ax=rax,
            histtype="errorbar",
            color=c,
            capsize=4,
        )

    ax.set_ylabel("Events [A.U.]")
    ax.legend()
    ax.set_title(label)

    rax.legend()
    rax.grid(axis="y")
    rax.set_xlabel(r"$m_{HH}$ [GeV]")
    rax.set_ylabel("Triggered / Preselection")
    rax.set_ylim(0, 1)

    hep.cms.label(ax=ax, data=False, year=2022, com="13.6")

    plt.savefig(plot_dir / f"mhh_{cat}.pdf", bbox_inches="tight")
    plt.show()

4b triggers

In [ ]:
df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"]

In [ ]:
bbbb_triggered = (
    df4b["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
    | df4b["HLT_AK8PFJet425_SoftDropMass40"][0]
)
higgs = utils.make_vector(df4b, name="GenHiggs")
mhh = (higgs[:, 0] + higgs[:, 1]).mass
weights = df4b["weight"][0]

In [ ]:
plt.rcParams.update({"font.size": 14})

cat = "bbbb"
(mask, label, triggers) = (
    np.ones(len(df4b)).astype(bool),
    "HH4b",
    {"ParticleNetBB | PFJet425 ": bbbb_triggered},
)

fig, (ax, rax) = plt.subplots(
    2, 1, figsize=(12, 14), gridspec_kw=dict(height_ratios=[4, 1], hspace=0.07), sharex=True
)

bins = np.linspace(250, 1500, 30)
hists = {"Preselection": np.histogram(mhh[mask], bins=bins, weights=weights[mask])}
ratios = {}

hep.histplot(
    hists["Preselection"],
    yerr=False,
    label="Preselection",
    ax=ax,
)

colours = plt.cm.tab10.colors[1:]

for key, c in zip(triggers.keys(), colours):
    print(key)
    hists[key] = np.histogram(
        mhh[mask & triggers[key]], bins=bins, weights=weights[mask & triggers[key]]
    )
    ratios[key] = hists[key][0] / hists["Preselection"][0]

    hep.histplot(
        hists[key],
        yerr=False,
        label=key,
        ax=ax,
        color=c,
    )

    hep.histplot(
        (ratios[key], bins),
        yerr=False,
        label=key,
        ax=rax,
        histtype="errorbar",
        color=c,
        capsize=4,
    )

ax.set_ylabel("Events [A.U.]")
ax.legend()
ax.set_title(label)

rax.legend()
rax.grid(axis="y")
rax.set_xlabel(r"$m_{HH}$ [GeV]")
rax.set_ylabel("Triggered / Preselection")
rax.set_ylim(0, 1)

hep.cms.label(ax=ax, data=False, year=2022, com="13.6")

plt.savefig(plot_dir / f"mhh_{cat}.pdf", bbox_inches="tight")
plt.show()

In [ ]:
twoboosted = df4b["ak8FatJetPt"][1] > 300
print(np.mean(bbbb_triggered))
print(np.sum(bbbb_triggered * twoboosted) / np.sum((twoboosted)))